# Produce the various exclusion plots

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext line_profiler

In [3]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import pandas as pd
import pickle
import gc

from exp_analysis_class import full_likelihood, compute_likelihood_from_retrieved
from dark_nus_utils import load_datasets
from const import alphaQED
from analyses_dict import analyses
from parameters_dict import *
from dark_nus_utils import store_events_weights, retrieve_events_weights

from likelihood_contours_plot import *

In [4]:
hierarchy = 'light'
D_or_M = 'majorana'

save_folder = f'../fig/6_limit_visualisation/{hierarchy}_{D_or_M}/'
!mkdir -p $save_folder

In [5]:
# my_exp_analyses = load_datasets(hierarchies=['heavy', 'light'],
#                                   D_or_Ms=D_or_M,
#                                   fluxes=['FHC', 'RHC'],
#                                 timeit=True, 
#                                 direct_load_objects=False,
#                                 load_grid=False,
#                                 dump=True,
#                                 nentries=1000000)

In [6]:
my_exp_analyses = load_datasets(
                                hierarchies=hierarchy, 
                                D_or_Ms=D_or_M, 
                                fluxes=['FHC', 'RHC'],
                                timeit=True, 
                                direct_load_objects=True,
                                load_grid=False,
                                dump=False,
                                nentries=1000000)

light majorana FHC
Wall time: 4.531589508056641 s, CPU time: 0.4667719880000001
light majorana RHC
Wall time: 4.5213847160339355 s, CPU time: 0.45933431199999974


# $m_{Z^\prime} - \varepsilon$

In [ ]:
this_case = f'{hierarchy}_mz_epsilon'

In [ ]:
retrieved = {}
for nu_mode in analyses[likelihood_calculation_pars[this_case]['analysis_name']].keys():
    retrieved[nu_mode] = retrieve_events_weights(this_case, 
                        analysis_name=likelihood_calculation_pars[this_case]['analysis_name'], 
                        nu_mode=nu_mode)

In [ ]:
analysis_names = ['tpc_argon_only', 'tpc_pod_only', 'tpc', 'tpc_projection1', 'tpc_projection2']
likes = {}
mus = {}
sigmas2s = {}
for analysis_name in analysis_names:
    likes[analysis_name] = 0
    mus[analysis_name] = 0
    sigmas2s[analysis_name] = 0
    for nu_mode in analyses[analysis_name].keys():
        print(analysis_name, nu_mode)
        out = compute_likelihood_from_retrieved(retrieved[nu_mode], 
                                          my_exp_analyses[f'{hierarchy}_{D_or_M}_{nu_mode}'], 
                                          analyses[analysis_name][nu_mode], 
                                          like_normalized=True)
        likes[analysis_name] += out[0]
        mus[analysis_name] += out[1]
        sigmas2s[analysis_name] += out[2]
        del out
        gc.collect()
    likes[analysis_name] -= likes[analysis_name].min()

In [ ]:
likes['tpc_projection1_2'] = likes['tpc_projection1'] + likes['tpc_projection2']
likes['tpc_projection1_2'] -= likes['tpc_projection1_2'].min()
mus['tpc_projection1_2'] = mus['tpc_projection1'] + mus['tpc_projection2']
sigmas2s['tpc_projection1_2'] = sigmas2s['tpc_projection1'] + sigmas2s['tpc_projection2']

In [ ]:
colors = ['deepskyblue', 'blue', 'navy']
contours = {}
for i, analysis_name in enumerate(['tpc_argon_only', 'tpc_pod_only', 'tpc']):
    contours[analysis_name] = plt.contour(retrieved['FHC']['pars']['mz'], 
                                            retrieved['FHC']['pars']['epsilon'], 
                                            likes[analysis_name].T, 
                                  levels=[likelihood_levels_2d[0.9]], colors=[colors[i]])
plt.loglog()
plt.legend([cntr.legend_elements()[0][0] for cntr in contours.values()], 
           contours.keys(),
           frameon=False,
           loc='lower right')
plt.xlabel(r'$m_Z\prime$')
plt.ylabel(r'$\epsilon$')

In [ ]:
colors = ['deepskyblue', 'blue', 'navy', 'black']
contours = {}
for i, analysis_name in enumerate(['tpc', 'tpc_projection1', 'tpc_projection2', 'tpc_projection1_2']):
    contours[analysis_name] = plt.contour(retrieved['FHC']['pars']['mz'], 
                                            retrieved['FHC']['pars']['epsilon'], 
                                            likes[analysis_name].T, 
                                  levels=[likelihood_levels_2d[0.9], np.inf], colors=[colors[i]])
plt.loglog()
plt.legend([cntr.legend_elements()[0][0] for cntr in contours.values()], 
           contours.keys(),
           frameon=False,
           loc='lower right')
plt.xlabel(r'$m_Z\prime$')
plt.ylabel(r'$\epsilon$')

In [ ]:
set_plot_style()

In [ ]:
ax = set_canvas('mz_epsilon')
mz_epsilon_heavy_plot(ax, retrieved['FHC']['pars']['m4'], retrieved['FHC']['pars']['mz'])
ax.set_ylim(1e-5, 1e-1)

colors = ['orange', 'tomato']
linestyles = ['-', '--']
contours = {}
for i, analysis_name in enumerate(['tpc', 'tpc_projection1_2']):
    contours[analysis_name] = plt.contour(retrieved['FHC']['pars']['mz'], 
                                            retrieved['FHC']['pars']['epsilon'], 
                                            likes[analysis_name].T, 
                                  levels=[likelihood_levels_2d[0.9], np.inf], 
                                          colors=[colors[i]], linestyles=[linestyles[i]])
    plt.contourf(retrieved['FHC']['pars']['mz'], 
                                            retrieved['FHC']['pars']['epsilon'], 
                                            likes[analysis_name].T, 
                                  levels=[likelihood_levels_2d[0.9], np.inf], 
                                          colors=[colors[i]], alpha=0.05)
plt.legend([cntr.legend_elements()[0][0] for cntr in contours.values()], 
           contours.keys(),
           frameon=False,
           loc='lower right')
plt.savefig(save_folder + this_case + '.pdf', bbox_inches='tight')

# $m_N - |U_{\mu N}|^2$

In [7]:
this_case = f'{hierarchy}_m4_Umu4_2'

In [8]:
retrieved = {}
for nu_mode in analyses[likelihood_calculation_pars[this_case]['analysis_name']].keys():
    retrieved[nu_mode] = retrieve_events_weights(this_case, 
                        analysis_name=likelihood_calculation_pars[this_case]['analysis_name'], 
                        nu_mode=nu_mode)

In [11]:
nu_mode = 'FHC'
analysis_name = 'nueccqe_fgd'

In [8]:
analysis_names = [
                    'tpc_light',
                    'tpc_light_projection1',
                    'tpc_light_projection2',
                    'nueccqe_fgd',
                    'single_photon_fgd',
                    'nueccqe_fgd_projection1',
                    'single_photon_fgd_projection1',
                    'nueccqe_fgd_projection2',
                    'single_photon_fgd_projection2',
                  ]
likes = {}
mus = {}
sigmas2s = {}
for analysis_name in analysis_names:
    likes[analysis_name] = 0
    mus[analysis_name] = 0
    sigmas2s[analysis_name] = 0
    for nu_mode in analyses[analysis_name].keys():
        print(analysis_name, nu_mode)
        out = compute_likelihood_from_retrieved(retrieved[nu_mode], 
                                          my_exp_analyses[f'{hierarchy}_{D_or_M}_{nu_mode}'], 
                                          analyses[analysis_name][nu_mode], 
                                          like_normalized=True)
        likes[analysis_name] += out[0]
        mus[analysis_name] += out[1]
        sigmas2s[analysis_name] += out[2]
        del out
        gc.collect()
    likes[analysis_name] -= likes[analysis_name].min()

tpc_light FHC
tpc_light RHC
tpc_light_projection1 FHC
tpc_light_projection1 RHC
tpc_light_projection2 FHC
tpc_light_projection2 RHC
nueccqe_fgd FHC


KeyboardInterrupt: 

In [ ]:
likes['tpc_light_projection1_2'] = likes['tpc_light_projection1'] + likes['tpc_light_projection1']
likes['tpc_light_projection1_2'] -= likes['tpc_light_projection1_2'].min()
likes['nueccqe_fgd_projection1_2'] = likes['nueccqe_fgd_projection1'] + likes['nueccqe_fgd_projection1']
likes['nueccqe_fgd_projection1_2'] -= likes['nueccqe_fgd_projection1_2'].min()
likes['single_photon_fgd_projection1_2'] = likes['single_photon_fgd_projection1'] + likes['single_photon_fgd_projection1']
likes['single_photon_fgd_projection1_2'] -= likes['single_photon_fgd_projection1_2'].min()

likes['final_projection1_2'] = likes['tpc_light_projection1_2'] + likes['single_photon_fgd_projection1_2']
likes['final_projection1_2'] -= likes['final_projection1_2'].min()

In [ ]:
colors = ['deepskyblue', 'blue', 'navy']
contours = {}
for i, analysis_name in enumerate(['tpc_light', 'nueccqe_fgd', 'single_photon_fgd']):
    contours[analysis_name] = plt.contour(retrieved['FHC']['pars']['m4'], 
                                            retrieved['FHC']['pars']['Umu4_2'], 
                                            likes[analysis_name].T, 
                                  levels=[likelihood_levels_2d[0.9]], colors=[colors[i]])
plt.loglog()
plt.legend([cntr.legend_elements()[0][0] for cntr in contours.values()], 
           contours.keys(),
           frameon=False,
           loc='lower right')
plt.xlabel(r'$m_{N}$ [GeV]')
plt.ylabel(r'$|U_{\mu N}|^2$')

In [ ]:
colors = ['deepskyblue', 'blue', 'navy']
contours = {}
for i, analysis_name in enumerate(['tpc_light_projection1_2', 'nueccqe_fgd_projection1_2', 'single_photon_fgd_projection1_2',]):
    contours[analysis_name] = plt.contour(retrieved['FHC']['pars']['m4'], 
                                            retrieved['FHC']['pars']['Umu4_2'], 
                                            likes[analysis_name].T, 
                                  levels=[likelihood_levels_2d[0.9]], colors=[colors[i]])
plt.loglog()
plt.legend([cntr.legend_elements()[0][0] for cntr in contours.values()], 
           contours.keys(),
           frameon=False,
           loc='lower right')
plt.xlabel(r'$m_{N}$ [GeV]')
plt.ylabel(r'$|U_{\mu N}|^2$')

In [ ]:
colors = ['deepskyblue', 'blue', 'navy', 'black']
contours = {}
for i, analysis_name in enumerate(['tpc', 'tpc_projection1', 'tpc_projection2', 'tpc_projection1_2']):
    contours[analysis_name] = plt.contour(retrieved['FHC']['pars']['m4'],
                                            retrieved['FHC']['pars']['Umu4_2'],
                                            likes[analysis_name].T, 
                                  levels=[likelihood_levels_2d[0.9], np.inf], colors=[colors[i]])
plt.loglog()
plt.legend([cntr.legend_elements()[0][0] for cntr in contours.values()], 
           contours.keys(),
           frameon=False,
           loc='upper center')
plt.set_xlabel(r'$m_{N}$ [GeV]')
plt.set_ylabel(r'$|U_{\mu N}|^2$')

In [ ]:
set_plot_style()

In [ ]:
from likelihood_contours_plot import m4_Umu4_2_heavy_plot

In [ ]:
ax = set_canvas('m4_Umu4_2')
m4_Umu4_2_light_plot(ax)
# ax.set_ylim(1e-10, 1)
ax.legend()
# colors = ['orange', 'tomato']
# linestyles = ['-', '--']
# contours = {}
# for i, analysis_name in enumerate(['tpc', 'tpc_projection1_2']):
#     contours[analysis_name] = plt.contour(retrieved['FHC']['pars']['m4'], 
#                                             retrieved['FHC']['pars']['Umu4_2'], 
#                                             likes[analysis_name].T, 
#                                   levels=[likelihood_levels_2d[0.9], np.inf], 
#                                           colors=[colors[i]], linestyles=[linestyles[i]])
#     plt.contourf(retrieved['FHC']['pars']['m4'], 
#                                             retrieved['FHC']['pars']['Umu4_2'], 
#                                             likes[analysis_name].T, 
#                                   levels=[likelihood_levels_2d[0.9], np.inf], 
#                                           colors=[colors[i]], alpha=0.05)
# plt.legend([cntr.legend_elements()[0][0] for cntr in contours.values()], 
#            contours.keys(),
#            frameon=False,
#            loc='lower left')
# plt.savefig(save_folder + this_case + '.pdf', bbox_inches='tight')